In [1]:
import os 
import sys
from os import listdir
from os.path import isfile, join
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import trange, tqdm
import skimage
from skimage import io, color, exposure
from skimage.util import img_as_ubyte
from skimage.transform import resize
from sklearn.preprocessing import MinMaxScaler
import matplotlib
from pathlib import Path
import re
import h5py
import cv2 as cv
from scipy import signal
import scipy.ndimage
import seaborn as sns
import pandas as pd
import scanpy as sc
import anndata as ad
import cv2

from matplotlib.pyplot import rc_context
sc.settings.verbosity = 3


In [2]:
p_dir = (Path().cwd().parents[0]).absolute()
data_dir = p_dir / 'data'

In [3]:
%load_ext autoreload
%autoreload 2

module_path = str(p_dir / "src")

if module_path not in sys.path:
    sys.path.append(module_path)

import utils as my_utils

# Read images

In [4]:
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm
fontprops = fm.FontProperties(size=50)

h5_data_dir = p_dir / 'data' / 'h5'

def get_imgs(experiment, name):
    with h5py.File(h5_data_dir / f'{experiment}.hdf5', 'r') as f:
        imgs = f[name][:]
        labels = list(f[name].attrs['labels'])
        fov = f[name].attrs['fov']
    return imgs, labels, fov

def contrast_stretching(img):
    # Contrast stretching
    p2, p98 = np.percentile(img, (0, 100))
    img_rescale = exposure.rescale_intensity(img, in_range=(p2, p98), out_range = (0, 255)).astype(np.uint8)
    return img_rescale
    
def get_mass(file):
    labels = ['total', 'rest']
    with open(file) as infile:
        next(infile)
        for line in infile:
            mass = float(line.split("\t")[1])
            mass = np.round(mass, 1)
            labels.append(mass)
    return labels


def get_imgs_index(imgs, index_list):
    
    imgs_subset = imgs[index_list, :,:]
    img_combined = np.sum(imgs_subset, axis=0)
    return img_combined


In [5]:
experiment = 'liver'

ts_data_dir = p_dir / 'data' / 'tof-sims' / experiment
datasets = listdir(ts_data_dir)
datasets = [dataset for dataset in datasets if 'auto' in dataset]

# Check intensity cells 

In [6]:
from skimage.filters import threshold_otsu
from skimage import measure
from collections import defaultdict
from statannotations.Annotator import Annotator
from scipy import ndimage
sns.set_context("talk", font_scale=2)

In [7]:
datasets

['auto_CPt1a-2030_1',
 'auto_CPt1a-2030_2',
 'auto_CPt1a-2837_1',
 'auto_CPt1a-2837_2',
 'auto_CPt1a-2837_3',
 'auto_VP16-cont2_1',
 'auto_VP16-cont2_2',
 'auto_VP16-cont2_3',
 'auto_VP16-Exp1_1',
 'auto_VP16-Exp1_2',
 'auto_VP16-Exp1_3']

In [17]:
df_prop = pd.concat(df_list)

In [16]:
info = defaultdict(list)
t = ['KO', 'KO', 'cont', 'cont', 'cont', 'cont', 'cont', 'cont', 'VP16', 'VP16', 'VP16']
df_list = []

for i, dataset in enumerate(datasets):
    # Read images and get PO3 channel
    print(dataset)

    imgs, labels, fov = get_imgs(experiment, dataset)
    img_nuclei= imgs[74]
    
    thresh = threshold_otsu(img_nuclei)
    binary = img_nuclei > thresh
    # Per cell intensity
    labeled_binary,_ = ndimage.label(binary)

    # Per mask region intensity
    subset = np.arange(2,len(labels))
    imgs_subset = imgs[subset]
    labels_subset = [labels[i] for i in subset]
    for j, img in enumerate(imgs_subset):
        info['Type'].append(t[i])
        info['Channel'].append(f'{float(labels_subset[j]):.0f}')
        info['Image mean intensity'].append(img.mean())
        a = np.ma.array(img, mask=binary)
        info['Cell mean intensity'].append(a.mean())
        a = np.ma.array(img, mask=~binary)
        info['ECM mean intensity'].append(a.mean())
        
        df_prop = pd.DataFrame(
            measure.regionprops_table(labeled_binary, img,  properties=['label', 'mean_intensity'])
        )
        df_prop['Channel'] = f'{float(labels_subset[j]):.0f}'
        df_prop['Dataset'] = dataset
        df_prop['Type'] = t[i]
        
    df_prop = df_prop.pivot_table(index=['Dataset', 'label', 'Type'], columns=['Channel'], values='mean_intensity')
    
    df_list.append(df_prop)

auto_CPt1a-2030_1
auto_CPt1a-2030_2
auto_CPt1a-2837_1
auto_CPt1a-2837_2
auto_CPt1a-2837_3
auto_VP16-cont2_1
auto_VP16-cont2_2
auto_VP16-cont2_3
auto_VP16-Exp1_1
auto_VP16-Exp1_2
auto_VP16-Exp1_3


In [22]:
df_prop

,label,mean_intensity,Channel,Dataset,Type
0,1,11.000000,1,auto_CPt1a-2030_1,KO
1,2,13.000000,1,auto_CPt1a-2030_1,KO
2,3,14.000000,1,auto_CPt1a-2030_1,KO
3,4,15.000000,1,auto_CPt1a-2030_1,KO
4,5,17.121951,1,auto_CPt1a-2030_1,KO
...,...,...,...,...,...
6721,6722,0.000000,413,auto_VP16-Exp1_3,VP16
6722,6723,0.000000,413,auto_VP16-Exp1_3,VP16
6723,6724,0.000000,413,auto_VP16-Exp1_3,VP16
6724,6725,0.000000,413,auto_VP16-Exp1_3,VP16


In [25]:
df_prop1 = df_prop.pivot_table(index=['Dataset', 'label', 'Type'], columns=['Channel'], values='mean_intensity').reset_index()

In [26]:
df_prop1

Channel,Dataset,label,Type,1,10,100,101,102,103,104,...,90,91,92,93,94,95,96,97,98,99
0,auto_CPt1a-2030_1,1,KO,11.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.00000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,3.000000,1.000000,1.000000
1,auto_CPt1a-2030_1,2,KO,13.000000,0.0,0.000000,0.000000,1.00000,0.000000,0.000000,...,2.00000,0.000000,1.000000,4.000000,0.000000,0.000000,0.000000,3.000000,6.000000,0.000000
2,auto_CPt1a-2030_1,3,KO,14.000000,0.0,2.000000,2.000000,0.00000,0.000000,0.000000,...,0.00000,1.000000,0.000000,2.000000,0.000000,1.000000,2.000000,8.000000,2.000000,1.000000
3,auto_CPt1a-2030_1,4,KO,15.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,...,1.50000,1.000000,0.000000,2.500000,0.500000,1.500000,2.500000,6.000000,3.000000,2.000000
4,auto_CPt1a-2030_1,5,KO,17.121951,0.0,0.390244,0.512195,1.02439,0.487805,0.292683,...,1.02439,1.902439,0.658537,1.487805,0.317073,1.365854,1.878049,5.341463,2.073171,1.390244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66297,auto_VP16-cont2_3,5448,cont,32.000000,0.0,0.000000,0.000000,1.00000,0.000000,0.000000,...,1.00000,3.000000,0.000000,0.000000,0.000000,1.000000,1.000000,11.000000,1.000000,2.000000
66298,auto_VP16-cont2_3,5449,cont,19.000000,0.0,1.000000,0.000000,0.00000,1.000000,0.000000,...,0.00000,2.000000,1.000000,3.000000,0.000000,2.000000,3.000000,1.000000,1.000000,3.000000
66299,auto_VP16-cont2_3,5450,cont,21.000000,0.0,0.000000,1.000000,0.00000,0.000000,0.000000,...,2.00000,3.000000,1.000000,0.000000,2.000000,0.000000,1.000000,4.000000,4.000000,2.000000
66300,auto_VP16-cont2_3,5451,cont,27.000000,0.0,0.000000,1.000000,0.00000,2.000000,0.000000,...,1.00000,3.000000,0.000000,1.000000,0.000000,0.000000,2.000000,5.000000,3.000000,4.000000
